In [ ]:
import xarray as xr

def grid_mapping(ruc_data_path, satellite_data, surface_data, radar_data, pireps, cali_data):
    """
    Maps various data sources onto the RUC model grid.

    Args:
        ruc_data_path (str): Path to the RUC model data file (e.g., NetCDF).
        satellite_data (xarray.Dataset): Satellite data containing cloud information.
        surface_data (list): List of surface observation dictionaries (METARs).
        radar_data (xarray.DataArray): Radar reflectivity data.
        pireps (list): List of PIREP dictionaries.
        lightning_data (list): List of lightning strike dictionaries.

    Returns:
        xarray.Dataset: Dataset containing all data mapped to the RUC grid.
    """

    # Load RUC data and extract relevant variables
    ruc_data = xr.open_dataset(ruc_data_path)
    temperature = ruc_data['temperature']
    relative_humidity = ruc_data['relative_humidity']
    vertical_velocity = ruc_data['vertical_velocity']
    geopotential_height = ruc_data['geopotential_height']
    # ... (add other RUC variables as needed)

    # --- Satellite Data Mapping ---
    # Assuming satellite_data is already on a suitable grid, perform any necessary interpolation or regridding here. 
    # For example, using xESMF package:
    # regridder = xe.Regridder(satellite_data, ruc_data, 'bilinear')
    # satellite_data_regridded = regridder(satellite_data)
    # Extract cloud information from the regridded satellite data (e.g., cloud top temperature, cloud mask)

      # --- Surface Data Mapping ---
    # Implement the concentric-circle approach to map surface observations to the RUC grid boxes. 
    # Create new variables in the RUC dataset to store ceiling height, precipitation occurrence, and precipitation type.

    # --- Radar Data Mapping ---
    # Assuming radar_data is already on a suitable grid, perform any necessary interpolation or regridding.
    # Calculate the percentage of each RUC grid box filled with echoes exceeding the chosen dBZ threshold. 
    # Store this information in a new variable within the RUC dataset.

    # --- PIREP and Lightning Data Mapping ---
    # For each PIREP, find the nearest RUC grid points within the specified range (150 km horizontally, 300 m vertically) and 
    # store the PIREP information (e.g., icing severity) and distance information in the dataset.
    # For each lightning strike, find the nearest RUC grid points within the specified range (25 km) and 
    # store a flag indicating the presence of lightning in the dataset.

    # --- Combine all data into a single dataset ---
    # Add the processed satellite, surface, radar, PIREP, and lightning data to the RUC dataset.
    # ...

    return ruc_data  # Now containing all mapped data

In [1]:
import xarray as xr 
import s3fs
import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
import cartopy.feature as cfeature 


fs = s3fs.S3FileSystem(anon=True)
f = fs.open("s3://noaa-goes16/ABI-L2-ACHTM/2024/001/00/OR_ABI-L2-ACHTM1-M6_G16_s20240010000281_e20240010000338_c20240010001227.nc")
ds = xr.open_dataset(f) 

In [28]:
def calc_latlon(ds):
    # The math for this function was taken from 
    # https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm
    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection
    
    x,y = np.meshgrid(x,y)
    
    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat
    
    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2
    
    r_s = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
    
    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)
    
    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)
    
    ds = ds.assign_coords({
        "lat":(["y","x"],lat),
        "lon":(["y","x"],lon)
    })
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"
    return ds

In [29]:
def get_xy_from_latlon(ds, lats, lons):
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data
    
    x = ds.x.data
    y = ds.y.data
    
    x,y = np.meshgrid(x,y)
    
    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 
    
    return ((min(x), max(x)), (min(y), max(y)))

In [30]:
ds = calc_latlon(ds)

In [31]:
ds.coords

Coordinates:
  * y                   (y) float32 2kB 0.1198 0.1198 0.1197 ... 0.09195 0.0919
  * x                   (x) float32 2kB -0.014 -0.01394 ... 0.01389 0.01394
    t                   datetime64[ns] 8B ...
    y_image             float32 4B ...
    x_image             float32 4B ...
    local_zenith_angle  float32 4B ...
    solar_zenith_angle  float32 4B ...
    lat                 (y, x) float32 1MB 45.73 45.73 45.73 ... 32.32 32.32
    lon                 (y, x) float32 1MB -81.85 -81.82 -81.8 ... -69.54 -69.52

In [32]:
lats = (30, 55)
lons = (-152, -112)

In [33]:
((x1,x2), (y1, y2)) = get_xy_from_latlon(ds, lats, lons)


ValueError: min() arg is an empty sequence